## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif, *_eyes_closed-raw.fif, *-epo.fif
####  Output: *_label_ts.pkl

In [1]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
from eeg_toolkit import utils, preprocess
from eeg_toolkit import source_localization as sl

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


### Paths

In [2]:
# Time window for source localization computation
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, always true unless you have subject-specific MRI and know what you're doing
average_dipoles = True

# Save inverse. True if you want to plot the brain model with AEC connectivity
save_inv = True

[-2.5,0.0,2.5]


In [3]:
# Read in processed raw data
data_dir = Path("../../Data")
processed_data_path = data_dir / 'Processed Data'
epo_path = data_dir / time_win_path

### Subject IDs

In [4]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_dir),'sub_ids.txt')
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [5]:
# All CP and HC subs
sub_ids_CP = [  
    '018',  
    '020',  
    '021',  
    '027',  
    '031',  
    '032',  
    '033',  
    '035',  
    '036',  
    '037',  
    '038',  
    '040',  
    '043',  
    '044',  
    '048',  
    '049',  
    '050',  
    '057',  
    '059',
]

sub_ids_HC = [
    'C10',  
    'C11',  
    'C12',  
    'C13',  
    'C14',  
    'C15',  
    'C16',  
    'C17',  
    'C18',  
    'C19',  
    'C2.',  
    'C24',  
    'C25',  
    'C26',  
    'C27',  
    'C3.',  
    'C6.',  
    'C7.',  
    'C9.'   
]
 

# CP Subs with widespread pain
sub_ids_WSP = [
    '018',
    '022',
    '024',
    '031',
    '032',
    '034',
    '036',
    '039',
    '040',
    '045',
    '046',
    '052',
]

# CP Subs with localized pain
sub_ids_LP = [  
    '020',  
    '021',  
    '023',  
    '029',  
    '031',  
    '037',  
    '041',  
    '042',  
    '048',  
    '049',  
    '050',  
    '056'  
]  

 
sub_ids = sub_ids_CP + sub_ids_HC + sub_ids_WSP + sub_ids_LP

# keep only unique
sub_ids = list(set(sub_ids))

In [6]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 31
Controls: 19
Total: 50


#### Look for subjects who do not have EO or EC cropped data

In [7]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

for sub_id in sub_ids:
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_closed-raw.fif")):
        no_eyes_closed.append(sub_id)
        print(f"Subject: {sub_id} missing eyes closed data")
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_open-raw.fif")):
        no_eyes_open.append(sub_id)
        print(f"Subject: {sub_id} missing eyes open data")
        
print(no_eyes_open)
print(no_eyes_closed)

[]
[]


In [8]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
            #  # CONTROLS
            #  lateral occipital
            #  'lateraloccipital-lh', # Left Visual Cortex
            #  'superiortemporal-lh', # Left Auditory Cortex
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh', # Right Medial-OFC
            #  # CONTROLS
            #  'lateraloccipital-rh', # Right Visual Cortex
            #  'superiortemporal-rh', # Right Auditory Cortex
]

### Compute STCs

In [9]:
sub_ids = ['046']

In [11]:
nan_subjects=[]

methods = [
    'MNE',
    # 'dSPM',
    ]

for sub_id in sub_ids: 
    # Check if eyes open or eyes closed need to be computed
    EO_bool = False if sub_id in no_eyes_open else True
    if not EO_bool:
        print(f"Subject: {sub_id} missing eyes open data")
        break
    
    for save_stc_mat in [
        False,
        # True,
        ]:    
        for method in methods:
            print(f"Source Localization Method: {method}")                          

            # Paths
            stc_path = (data_dir / f'Source Time Courses ({method}) (MAT)' 
                        if save_stc_mat 
                        else data_dir / f'Source Time Courses ({method}) 046')
                           
            EO_resting_save_path = stc_path / "Eyes Open"
            EC_resting_save_path = stc_path / "Eyes Closed"
            # zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            save_paths = [EC_resting_save_path,EO_resting_save_path,zscored_epochs_save_path] 
            [os.makedirs(path) for path in save_paths if not os.path.exists(path)]
            
            # No resting if exporting to MAT
            if save_stc_mat:
                Epochs_bool = True
                EC_bool = False
                EO_bool = False
            else:
                Epochs_bool = True

            # Compute source localization for subject and save 
            label_ts_All_Conds, sub_id_if_nan = sl.to_source(sub_id,
                                                             processed_data_path,
                                                             zscored_epochs_save_path,
                                                             EC_resting_save_path,
                                                             EO_resting_save_path,
                                                             roi_names,
                                                             times_tup,
                                                             method=method,
                                                             return_zepochs= Epochs_bool,
                                                             return_EC_resting = False,
                                                             return_EO_resting = EO_bool,
                                                             average_dipoles=average_dipoles,
                                                             save_stc_mat=save_stc_mat,
                                                             save_inv=save_inv,)
            utils.clear_display()
            
            # add nan subjects to list if not None
            if sub_id_if_nan is not None:
                nan_subjects.append(sub_id_if_nan)

Saved 046_inv.pkl to ../../Data/Source Time Courses (MNE) 046/zscored_Epochs/5_sec_time_window.


In [ ]:
nan_subjects